<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M3_agent_2_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Additional Agents in LangChain

LangChain offers a variety of agents designed to handle different types of tasks and interactions. In this notebook, we'll explore several additional agents to showcase the versatility and power of LangChain.

## Setup

First, let's install the necessary packages and import the required modules.

In [1]:
# Install necessary packages
!pip install langchain openai pandas faiss-cpu duckduckgo-search --q
!pip install -qU langchain openai sqlalchemy wikipedia langchain_community --q
# Note: Restart the kernel after installation.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


### Import Modules

In [2]:
from getpass import getpass

OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

Enter your OpenAI API key:··········


In [3]:
!pip install -qU langchain_experimental --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 3.6 MB/s eta 0:00:00


In [41]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_experimental.agents import create_pandas_dataframe_agent # Import from the correct location
from langchain.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import VectorStoreToolkit
import pandas as pd
import openai

# Define the LLM (Language Model) using OpenAI
from langchain.llms import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

### Helper Functions and Data

We'll define a helper function `count_tokens` and some sample data required for the agents.

In [6]:
def count_tokens(agent, prompt):
    response = agent.run(prompt)
    # Simplified token count (approximate)
    tokens = len(prompt.split()) + len(response.split())
    print(response)
    print(f"Spent a total of {tokens} tokens")
    return response

# Sample movie data for Pandas DataFrame Agent
movies_data = [
    {'title': 'Inception', 'director': 'Christopher Nolan', 'rating': 8.8},
    {'title': 'Interstellar', 'director': 'Christopher Nolan', 'rating': 8.6},
    {'title': 'The Dark Knight', 'director': 'Christopher Nolan', 'rating': 9.0},
    {'title': 'Pulp Fiction', 'director': 'Quentin Tarantino', 'rating': 8.9},
    {'title': 'Django Unchained', 'director': 'Quentin Tarantino', 'rating': 8.4}
]

---

## 1. Structured Chat Agent

**Description:**
The Structured Chat Agent leverages OpenAI's structured output capabilities to interact with tools in a more controlled and predictable manner. It uses the `StructuredChatAgent` to define the structure of the conversation, ensuring that the agent's responses adhere to a specific format.

**Use Cases:**
- When you need the agent to produce outputs in a specific structure or format.
- Useful in applications where the consistency of the response format is critical, such as generating JSON responses or filling out forms.

**Example:**

In [48]:
# Structured Chat Agent Example
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate


# Define a custom prompt
prompt = PromptTemplate(
    input_variables=["input"],
    template="""
    You are a structured chat agent. Please respond to the user's query in the following JSON format:
    {{
        "Price": "The price here",
        "summary": "summary of the recent performance" # Provide a brief summary
    }}

    Here are some examples:

    User Query: What is the price of gold and what is the recent performance?
    Response:
    {{
        "Price": "$1,900",
        "summary": "Gold prices have been relatively stable recently."
    }}

    User Query: I want to know about Tesla stock, how is it doing?
    Response:
    {{
        "Price": "$250",
        "summary": "Tesla stock has experienced significant growth this year."
    }}

    User Query: {input}
    Response:
    JSON:
        {{
        "Price": "The price here",
        "summary": "summary of the recent performance" # Provide a brief summary
    }}
    """ # Added "JSON:" to explicitly instruct the LLM to return JSON output
)

# Define the tool
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for answering questions about current events or the current state of the world."
    )
]

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the agent
structured_chat_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    max_iterations=2,
    agent_kwargs={'prompt': prompt.template}
)

# Test the agent
result = count_tokens(
    structured_chat_agent,
    "What is the current price of Bitcoin, and can you provide a brief summary of its recent performance?"
)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search",
  "action_input": "Bitcoin price"
}
```


Observation: Live price of Bitcoin is $98,099.82 with a market cap of $1,941.05B USD. Discover current price, trading volume, chart history, and more. Bitcoin (BTC) price, live charts, news and more. Bitcoin to USD price is updated in real time. Learn about Bitcoin, receive market updates and more. The live price of Bitcoin is $ 98,641.38 per (BTC / USD) with a current market cap of $ 1,951.73B USD. 24-hour trading volume is $ 73.56B USD. BTC to USD price is updated in real-time. Bitcoin is -0.38% in the last 24 hours with a circulating supply of 19.79M. The Kitco Bitcoin price index provides the latest Bitcoin price in US Dollars using an average from the world's leading exchanges. Bitcoin is climbing this week. The current price of Bitcoin is $98,217.41 per BTC. With a circulating supply of 19,786,887 BTC, it means that Bitcoin has a total market cap of $1,939,511,204

In [49]:
result

'The current price of Bitcoin is $98,099.82 with a market cap of $1,941.05B USD. In the last 24 hours, Bitcoin has seen a 10.51% increase in trading volume and is currently at -0.38% in the market. Its circulating supply is 19.79M and its total market cap is $1,939,511,204,287.17.'

**Sample Output:**

```
> Entering new AgentExecutor chain...
...

Final Answer: {
  "price": "$34,000",
  "summary": "Bitcoin has seen a steady increase in value over the past month due to market optimism."
}

> Finished chain.
Spent a total of 350 tokens
```

---

## 2. OpenAI Functions Agent

**Description:**
With the introduction of OpenAI's function calling capabilities, the OpenAI Functions Agent can perform actions by calling functions defined in code. This allows for more robust integrations with external systems and can handle complex tasks that require precise operations.

**Use Cases:**
- When you need the agent to interact with APIs or execute specific functions based on user queries.
- Useful for applications like booking appointments, performing calculations, or manipulating data.

**Example:**

In [ ]:
# OpenAI Functions Agent Example for CalendarManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI


# Define custom functions
def schedule_event(event_details: str) -> str:
    """
    Schedules a new calendar event based on the provided details.

    Args:
        event_details (str): A description of the event to be scheduled.

    Returns:
        str: Confirmation message for the scheduled event.
    """
    # In a real application, you would integrate with a calendar API here.
    # For simplicity, we'll just return a confirmation message.
    return f"Event scheduled: {event_details}"

def list_events(unused_input: str = "") -> str: # Modified function signature
    """
    Lists all scheduled calendar events.

    Args:
        unused_input (str, optional): Unused input string. Defaults to "".

    Returns:
        str: A list of scheduled events.
    """
    # In a real application, you would retrieve events from a calendar API.
    # For simplicity, we'll return a static list.
    return "You have the following events scheduled:\n1. Team Meeting at 10 AM\n2. Doctor Appointment at 3 PM"

# Define Tools for the agent
tools = [
    Tool(
        name="ScheduleEvent",
        func=schedule_event,
        description="Schedules a new calendar event based on the provided details."
    ),
    Tool(
        name="ListEvents",
        func=list_events,
        description="Lists all your scheduled calendar events."
    )
]

# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

# Initialize the agent with the defined tools
calendar_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    max_iterations=3
)

# Test the agent by running a query
def main():
    # Example queries
    queries = [
        "Schedule a meeting with John tomorrow at 2 PM.",
        "What are my events for today?"
    ]

    for query in queries:
        print(f"\nUser Query: {query}")
        result = calendar_agent.run(query)
        print("Agent Response:", result)

if __name__ == "__main__":
    main()



User Query: Schedule a meeting with John tomorrow at 2 PM.


> Entering new AgentExecutor chain...

Invoking: `ScheduleEvent` with `Meeting with John at 2 PM tomorrow`


Event scheduled: Meeting with John at 2 PM tomorrowI have successfully scheduled a meeting with John tomorrow at 2 PM.

> Finished chain.
Agent Response: I have successfully scheduled a meeting with John tomorrow at 2 PM.

User Query: What are my events for today?


> Entering new AgentExecutor chain...

Invoking: `ListEvents` with `today`


You have the following events scheduled:
1. Team Meeting at 10 AM
2. Doctor Appointment at 3 PMYou have the following events scheduled for today:
1. Team Meeting at 10 AM
2. Doctor Appointment at 3 PM

> Finished chain.
Agent Response: You have the following events scheduled for today:
1. Team Meeting at 10 AM
2. Doctor Appointment at 3 PM


---

## 3. Pandas DataFrame Agent

**Description:**
The Pandas DataFrame Agent is designed to interact with Pandas DataFrames, allowing the LLM to perform data analysis tasks directly on tabular data. It can answer questions about the data, perform aggregations, and generate plots.

**Use Cases:**
- When working with datasets and you want to query or analyze the data using natural language.
- Useful in data analysis, reporting, and exploratory data analysis tasks.

**Example:**

Suppose we have a DataFrame containing movie data.

In [ ]:
!pip install langchain_openai --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00


In [ ]:
# Pandas DataFrame Agent Example
import pandas as pd

# Sample movie data
df = pd.DataFrame(movies_data)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
import pandas as pd

agent_executor = create_pandas_dataframe_agent(
    llm,
    df,
    allow_dangerous_code=True,
    verbose=True,
    max_iterations=2
)

In [ ]:
# Test the agent
result = count_tokens(
    agent_executor,
    "What is the average rating of movies directed by Christopher Nolan?"
)



> Entering new AgentExecutor chain...
Thought: To find the average rating of movies directed by Christopher Nolan, we need to filter the dataframe for movies directed by him and then calculate the average of their ratings.
Action: python_repl_ast
Action Input: df[df['director'] == 'Christopher Nolan']['rating'].mean()8.799999999999999I now know the final answer
Final Answer: The average rating of movies directed by Christopher Nolan is 8.8

> Finished chain.
The average rating of movies directed by Christopher Nolan is 8.8
Spent a total of 22 tokens


**Sample Output:**

```
> Entering new AgentExecutor chain...
...

Final Answer: The average rating of movies directed by Christopher Nolan is 8.8.

> Finished chain.
Spent a total of 220 tokens
```

---

## 4. Vector Store Agent

**Description:**
The Vector Store Agent interacts with vector databases to perform semantic searches based on embeddings. This is particularly useful when dealing with unstructured text data.

**Use Cases:**
- When you need to find documents or pieces of information that are semantically similar to a query.
- Useful in applications like document retrieval, question answering over a knowledge base, and content recommendation.

**Example:**

In [ ]:
# Install necessary packages
# !pip install langchain openai chromadb sentence_transformers --quiet

# Import necessary modules
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import initialize_agent, AgentType
from langchain.agents.agent_toolkits import VectorStoreToolkit, VectorStoreInfo
from langchain.llms import OpenAI
import os

# Sample documents
documents = [
    "The Shawshank Redemption is a movie about hope and friendship.",
    "The Godfather focuses on the Italian-American mafia.",
    "Inception explores complex concepts of dreams within dreams."
]

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

# Create Chroma vector store from texts
vectorstore = Chroma.from_texts(
    texts=documents,
    embedding=embeddings,
    collection_name="movies_collection"
)

# Initialize the language model
llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

# Create VectorStoreInfo
vectorstore_info = VectorStoreInfo(
    name="movies_collection",
    description="A collection of movie descriptions",
    vectorstore=vectorstore
)

# Initialize the VectorStoreToolkit with the llm
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info, llm=llm)

# Initialize the agent
vectorstore_agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=2
)

# Define the count_tokens function
def count_tokens(agent, query):
    response = agent.run(query)
    token_count = len(response.split())
    print(f"Response: {response}")
    print(f"Token Count: {token_count}")
    return response

# Test the agent
result = count_tokens(
    vectorstore_agent,
    "Find a movie about dreams."
)




> Entering new AgentExecutor chain...
 I should use movies_collection since I only need the movie title.
Action: movies_collection
Action Input: "What is a movie about dreams?"
Observation:  Inception
Thought: I now know the final answer
Final Answer: Inception

> Finished chain.
Response: Inception
Token Count: 1


**Sample Output:**

```
> Entering new AgentExecutor chain...
...

Final Answer: "Inception" is a movie that explores complex concepts of dreams within dreams.

> Finished chain.
Spent a total of 280 tokens
```